# **NLP1_Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """ A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

Parameters:
C1 = 49000, cost coefficient for COLI 
C2 = 30, cost coefficient for PEPSA
P = $150, cost per hectolitre of syrup 
T = 5*8*7100, total production time available per week (unit: product units)
M = 0.5, minimum fraction of PEPSA in total production 

Decision variables: 
X1, number of COLI units produced
X2, number of PEPSA units produced

Objective function:
Maximize profit Z = (C1 * X1 - X1^2) + (C2 * X2 - 2 * X2^2) - P * ((X1/40) + (X2/20))

Constraints:
1) Time constraint: X1 + X2 ≤ T
2) Minimum PEPSA production constraint: X2 ≥ M * X1
3) Non-negativity constraints: X1, X2 ≥ 0

This mathematical optimization model will help the company in deciding the optimal number of COLI and PEPSA units to produce in order to maximize their weekly profits while meeting the production time and minimum PEPSA production constraints. The non-negativity constraints ensure that the number of units produced can't be negative.


### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
Markdown(response2.choices[0].message.content)

The Python Pyomo code for this optimization problem is as follows:

```python
from pyomo.environ import *

# Create a new model
model = ConcreteModel()

# Define the decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define the objective function
model.profit = Objective(expr = (49000*model.x1 - model.x1**2) + (30*model.x2 - 2*model.x2**2) - 150 * ((model.x1/40) + (model.x2/20)), sense=maximize)

# Define the constraints
model.time_constraint = Constraint(expr = model.x1 + model.x2 <= 5*8*7100)
model.pepsa_constraint = Constraint(expr = model.x2 >= 0.5 * model.x1)

# Create a solver
solver = SolverFactory('ipopt')

# Solve the model
solver.solve(model)

# Print the value of the decision variables at the optimal solution
print("number of COLI units produced=", model.x1())
print("number of PEPSA units produced=", model.x2())
print("Maximum profit=", model.profit())
```

The decision variables model.x1 and model.x2 represent the number of COLI and PEPSA units produced. 
The maximum profit equation is represented by the Objective function.
The maximum time and minimum PEPSA constraints are captured in the model.time_constraint and model.pepsa_constraint.
The 'ipopt' solver is used to solve the optimization model.

Please note that the 'pyomo.environ' library and the solver 'ipopt' need to be properly installed in your Python environment to perfectly run this code.

### **Run the code Generated by GPT4**

In [9]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 11.9 MB/s eta 0:00:00


In [11]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [15]:
from pyomo.environ import *

# Create a new model
model = ConcreteModel()

# Define the decision variables
model.x1 = Var(domain=NonNegativeReals)
model.x2 = Var(domain=NonNegativeReals)

# Define the objective function
model.profit = Objective(expr = (49000*model.x1 - model.x1**2) + (30*model.x2 - 2*model.x2**2) - 150 * ((model.x1/40) + (model.x2/20)), sense=maximize)

# Define the constraints
model.time_constraint = Constraint(expr = model.x1 + model.x2 <= 5*8*7100)
model.pepsa_constraint = Constraint(expr = model.x2 >= 0.5 * model.x1)

# Create a solver
solver = SolverFactory('knitro')

# Solve the model
solver.solve(model)

# Print the value of the decision variables at the optimal solution
print("number of COLI units produced=", model.x1())
print("number of PEPSA units produced=", model.x2())
print("Maximum profit=", model.profit())

number of COLI units produced= 16335.833333842567
number of PEPSA units produced= 8167.916666951235
Maximum profit= 400289176.04068875


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**